<a href="https://colab.research.google.com/github/HongYue1/LightNovel-Crawler-Colab/blob/main/lightnovel_crawler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# How to use:
### 1. Click on connect on the top right and wait untill it finishs connecting.
### 2. Before running a block, make sure to fill the required data first.
### 3. Run each block, wait for the block to finish before running the next block, the block finishes when you see a green check mark next to the run button.
###4. After uploading finish you will be given the link as an output.

#### <The archive and upload cells are From OneClickRun, all credit belongs to their repective owners.>

## ⏬ Install LightNovel-crawler

In [ ]:
# @markdown ## ⬅️ Click Here to install LightNovel-crawler
%%capture
!pip install -U lightnovel-crawler;

## ⏬ Download Novel

In [ ]:
#@markdown <h3>⬅️ Click Here to Download</h3>

def download():
   #@markdown <h4> Enter the Link:
   Link = ""#@param {type:"string"}

   #@markdown <h4> Enter the range of the chapters separted by a space or enter All to download all chapters.
   #@markdown <h5> Example to download chapters between 1 and 100 : 1 100
   Range = "All"#@param {type:"string"}

   #@markdown <h4> Choose a Format:
   #@markdown <h4> Not working for now, only epub will be downloaded.

   Format="epub"
   if(Range=="All"):
    !lncrawl --single -o "/content/novels" -s $Link --format epub --all -f --suppress --auto-proxy
   else:
     !lncrawl --single -o "/content/novels" -s $Link  --range $Range --format epub  -f --suppress --auto-proxy

download()

##<img src='https://raw.githubusercontent.com/biplobsd/OneClickRun/master/img/zip.png' height="30" alt="netdata"> Archive and Upload

In [ ]:
# ============================= FORM ============================= #
# @markdown ## ⬅️ Archive Files
%%capture

import os

# Specify the directory path
directory_path = '/content/novels/epub'

# Get the list of files in the directory
file_list = os.listdir(directory_path)

# Get the first file from the list
first_file = file_list[0]

# Split the file name and extension
file_name_without_extension = os.path.splitext(first_file)[0]


MODE = "ZIP" #@param ["ZIP", "TAR", "7Z"]
FILENAME = file_name_without_extension+'.zip'
PATH_TO_FILE = "/content/novels"
# @markdown #####   For no password leave it empty:
ARCHIVE_PASSWORD = "" #@param {type:"string"}

# option supports b k m g (bytes, kilobytes, megabytes, gigabytes)
SPLIT = "no"


compress = 9#@param  {type:"slider", min:0, max:9, step:0}

# ================================================================ #
from pathlib import PurePosixPath

pathList = PATH_TO_FILE.split('|')
if MODE == "ZIP":
    if not FILENAME:
      FILENAME = "/content/NEW_FILE.ZIP"
    if ARCHIVE_PASSWORD:
      passADD = f'--password "{ARCHIVE_PASSWORD}"'
    else:
      passADD = ''
    splitC = f"-s {SPLIT}" if not 'no' in SPLIT else ""
    for part in pathList:
      pathdic = PurePosixPath(part.strip())
      parent = pathdic.parent
      partName = pathdic.parts[-1]
      cmd = f'cd "{parent}" && zip {passADD} -{compress} {splitC} -v -r -u "{FILENAME}" "{partName}"'
      !$cmd
elif MODE == "TAR":
    if not FILENAME:
      FILENAME = "/content/NEW_FILE"
    cmd = f'GZIP=-{compress} tar -zcvf "{FILENAME}.tar.gz" {PATH_TO_FILE}'
    !$cmd
else:
    if not FILENAME:
        FILENAME = "/content/NEW_FILE"
    for part in pathList:
      pathdic = PurePosixPath(part.strip())
      parent = pathdic.parent
      partName = pathdic.parts[-1]
      cmd = f'cd "{parent}" && 7z a -mx={compress} "{FILENAME}.7z" "{partName}"'
      !$cmd


# 🔼 Upload

In [ ]:
#@markdown <h2>⬅️ Click Here to install MultiUpload requirements</h3>
import os, subprocess, json

wd = 'tools/multiupload'
progName = 'go-ul_linux_x64'
os.makedirs(wd, exist_ok=True)
try:
  if not os.path.exists(wd+'/'+progName):
    cmd = !cd $wd;wget https://github.com/bunnykek/go-upload/releases/download/tag/$progName
    cmd = !cd $wd;chmod 755 $progName
  if not cmd:
    print("Requirements installed successfully.")
  else:
    print(cmd, 'Requirements installed unsuccessfully', sep='\n')
except Exception as e:
  print('Error: ', e)

In [ ]:
#@markdown <h2>⬅️ Click Here to upload</h2>

#@markdown <h5> You can change the filehost, but transfer.sh is fast and easy.

FileHost = "transfersh" #@param ['catbox', 'fileio', 'filemail', 'gofile', 'krakenfiles', 'letsupload', 'megaup', 'mixdrop', 'pixeldrain', 'racaty', 'transfersh', 'uguu', 'wetransfer', 'workupload']
File_Path = "/content/" + file_name_without_extension + ".zip"
filehost, infile = FileHost, File_Path

message = """
Name : {0}
Host : {1}
URL  : {2}"""
cwd = 'tools/multiupload/'
def upload(filehost:str, filePath: str):
    subprocess.Popen(["./go-ul_linux_x64", filehost,
                     '-f', infile, '-j', 'response.json'],
                     cwd=cwd).wait()

    response = json.load(open(cwd+'response.json'))

    if not response['jobs'][-1]['ok']:
        print(f"{response['jobs'][-1]['error_text']}")
    else:
        print(message.format(response['jobs'][-1]['filename'], filehost, response['jobs'][-1]['url']))

upload(filehost, infile)

